<a href="https://colab.research.google.com/github/mcdodik20008/tg-bot/blob/master/notebooks/resume_kasphitskii_parashchuk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/mcdodik20008/tg-bot/refs/heads/master/datasets/combined/combined_df.csv

--2025-10-21 12:39:02--  https://raw.githubusercontent.com/mcdodik20008/tg-bot/refs/heads/master/notebooks/combined_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10732237 (10M) [text/plain]
Saving to: ‘combined_df.csv’

combined_df.csv     100%[===================>]  10.23M  32.5MB/s    in 0.3s    

2025-10-21 12:39:04 (32.5 MB/s) - ‘combined_df.csv’ saved [10732237/10732237]



In [ ]:
import pandas as pd
import numpy as np
import re
import math

In [ ]:
df = pd.read_csv('combined_df.csv')

In [ ]:
df.head()

,full_name,desired_position,salary,personal_info,location,employment_and_schedule,total_experience,last_position,last_job_company,responsibilities,key_skills,about_me,education,languages,link
0,Кандидат,Python-разработчик,Не указано,"Мужчина, 32 года","Москва , не готов к переезду , готов к команди...",Не указано,Опыт работы 1 год,Python-разработчик,Диджитал агенство Рядышком,"Создал API сервис, для проекта не имеющего ана...","Python, Django Framework, REST API, DjangoORM,...","Я решил перейти в сферу разработки, так как ме...","ГОУ СПО Строительный колледж №38 Туризм , Мене...","Русский — Родной, Английский — A2 — Элементарный",https://hh.ru/resume/79d001390007c9fa5b0039ed1...
1,Кандидат,Python-разработчик,150 000 ₽ на руки,"Мужчина, 26 лет","Москва , не готов к переезду , готов к команди...",Не указано,Опыт работы 2 года 3 месяца,Python-разработчик,Decurity,Основной задачей была разработка и поддержка с...,"Исполнительность, Асинхронное программирование...",Backend-разработчик с более чем двухлетним опы...,Московский государственный технический универс...,"Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/01e7303a00090f41c50039ed1...
2,Кандидат,Python-разработчик,160 000 ₽ на руки,"Мужчина, 28 лет","Москва , м. Новаторская , не готов к переезду ...",Не указано,Опыт работы 4 года 6 месяцев,Python-разработчик,"Вайлдберриз, ООО",Разрабатывал с коллегами следующие сервисы: Се...,"Python, Django Framework, SQL, Английский язык...",Более 4 лет опыта с python и 2 лет коммерческо...,Московский государственный институт международ...,"Русский — Родной, Английский — C1 — Продвинуты...",https://hh.ru/resume/f4f2bed50005a7d6eb0039ed1...
3,Кандидат,Python - разработчик,Не указано,"Мужчина, 26 лет","Москва , не готов к переезду , готов к команди...",Не указано,Опыт работы 5 лет 6 месяцев,Python-разработчик,"ООО ""МТС ДИДЖИТАЛ""","Разработка бекенда на фреймворке Flask, FastAP...","Python, SQL, Работа в команде, ООП, Django Fra...",Хайбулин Роман Олегович Дата рождения: 17.02.1...,Московский финансово-юридический университет М...,"Русский — Родной, Английский — A2 — Элементарный",https://hh.ru/resume/cd10011e00088d37c50039ed1...
4,Кандидат,Python-разработчик,Не указано,"Мужчина, 23 года","Ташкент , готов к переезду (Москва) , готов к ...",Не указано,Опыт работы 3 года,Разработчик,VK,1. Миграция инфраструктуры (Яндекс → VK) - Зад...,"Python, Telegram, TeamCity, Bash, Docker, Java...",Github: https://github.com/FPSarmin tg:@fpsarmin,"Национальный исследовательский университет ""Вы...","Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/7799fa7000090a8dd80039ed1...


In [ ]:
# Дропнули имя, не несет смысла
df.drop(columns=['full_name'], inplace=True)

In [ ]:
# Дропнули занятость, не заполнено практически нигде
df.drop(columns=['employment_and_schedule'], inplace=True)

In [ ]:
# Разделили последнее место работы
df[['last_position', 'last_job_company']] = df['last_job'].str.split(' в ', n=1, expand=True)

df.drop(columns=['last_job'], inplace=True)

In [ ]:
# Разделили пол и возраст
df[['gender', 'age']] = df['personal_info'].str.split(', ', n=1, expand=True)
df.drop(columns=['personal_info'], inplace=True)


df['age'] = df['age'].str.extract('(\\d+)').astype('Int64')

In [ ]:
def parse_experience(exp_str):
    if pd.isna(exp_str):
        return pd.NA

    text = str(exp_str).replace('Опыт работы', '').strip()

    years, months = 0, 0

    year_match = re.search(r'(\d+)\s*(?:год|года|лет)', text)
    if year_match:
        years = int(year_match.group(1))

    month_match = re.search(r'(\d+)\s*(?:месяц|месяца|месяцев)', text)
    if month_match:
        months = int(month_match.group(1))

    if months >= 6:
        years += 1

    return years

# Привели к числам опыт
df['experience_years'] = df['total_experience'].apply(parse_experience).astype('Int64')
df.drop(columns=['total_experience'], inplace=True)

In [ ]:
# Обработка зарплаты
def parse_salary(salary_str):
    if pd.isna(salary_str):
        return np.nan, np.nan

    numbers = re.findall(r'\d+', str(salary_str))
    if not numbers:
        return np.nan, np.nan

    amount = int("".join(numbers))

    salary_lower = str(salary_str).lower()
    currency = 'RUB'
    if '$' in salary_lower or 'usd' in salary_lower:
        currency = 'USD'
    elif '€' in salary_lower or 'eur' in salary_lower:
        currency = 'EUR'
    elif '₸' in salary_lower or 'kzt' in salary_lower:
        currency = 'KZT'

    return amount, currency

exchange_rates = { 'RUB': 1, 'USD': 95, 'EUR': 102, 'KZT': 0.2 }

df[['salary_amount', 'currency']] = df['salary'].apply(lambda x: pd.Series(parse_salary(x)))
df['salary_rub'] = df.apply(
    lambda row: row['salary_amount'] * exchange_rates.get(row['currency'], 0) if pd.notna(row['salary_amount']) else np.nan,
    axis=1
).astype('Int64')
df.drop(columns=['salary_amount', 'currency', 'salary'], inplace=True)

In [ ]:
# Обработка местоположения
def parse_location(location_str):
    if pd.isna(location_str):
        return np.nan, np.nan, np.nan

    parts = [p.strip() for p in location_str.split(',')]
    city = parts[0]
    can_move = np.nan
    can_trip = np.nan

    for part in parts[1:]:
        if 'переезду' in part:
            can_move = 'не готов' not in part
        if 'командировкам' in part:
            can_trip = 'не готов' not in part

    return city, can_move, can_trip

df[['city', 'can_move', 'can_trip']] = df['location'].apply(lambda x: pd.Series(parse_location(x)))

df['can_move'] = df['can_move'].astype('boolean')
df['can_trip'] = df['can_trip'].astype('boolean')
df.drop(columns=['location'], inplace=True)

In [ ]:
df['gender'] = df['gender'].apply(lambda x: 0 if x == 'Мужчина' else (1 if x == 'Женщина' else -1))

In [ ]:
df.head()

,desired_position,last_position,last_job_company,responsibilities,key_skills,about_me,education,languages,link,gender,age,experience_years,salary_rub,city,can_move,can_trip
0,Python-разработчик,Python-разработчик,Диджитал агенство Рядышком,"Создал API сервис, для проекта не имеющего ана...","Python, Django Framework, REST API, DjangoORM,...","Я решил перейти в сферу разработки, так как ме...","ГОУ СПО Строительный колледж №38 Туризм , Мене...","Русский — Родной, Английский — A2 — Элементарный",https://hh.ru/resume/79d001390007c9fa5b0039ed1...,0,32,1,<NA>,Москва,False,True
1,Python-разработчик,Python-разработчик,Decurity,Основной задачей была разработка и поддержка с...,"Исполнительность, Асинхронное программирование...",Backend-разработчик с более чем двухлетним опы...,Московский государственный технический универс...,"Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/01e7303a00090f41c50039ed1...,0,26,2,150000,Москва,False,True
2,Python-разработчик,Python-разработчик,"Вайлдберриз, ООО",Разрабатывал с коллегами следующие сервисы: Се...,"Python, Django Framework, SQL, Английский язык...",Более 4 лет опыта с python и 2 лет коммерческо...,Московский государственный институт международ...,"Русский — Родной, Английский — C1 — Продвинуты...",https://hh.ru/resume/f4f2bed50005a7d6eb0039ed1...,0,28,5,160000,Москва,False,True
3,Python - разработчик,Python-разработчик,"ООО ""МТС ДИДЖИТАЛ""","Разработка бекенда на фреймворке Flask, FastAP...","Python, SQL, Работа в команде, ООП, Django Fra...",Хайбулин Роман Олегович Дата рождения: 17.02.1...,Московский финансово-юридический университет М...,"Русский — Родной, Английский — A2 — Элементарный",https://hh.ru/resume/cd10011e00088d37c50039ed1...,0,26,6,<NA>,Москва,False,True
4,Python-разработчик,Разработчик,VK,1. Миграция инфраструктуры (Яндекс → VK) - Зад...,"Python, Telegram, TeamCity, Bash, Docker, Java...",Github: https://github.com/FPSarmin tg:@fpsarmin,"Национальный исследовательский университет ""Вы...","Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/7799fa7000090a8dd80039ed1...,0,23,3,<NA>,Ташкент,True,True


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835 entries, 0 to 4834
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   desired_position  4835 non-null   object 
 1   last_position     4835 non-null   object 
 2   last_job_company  4475 non-null   object 
 3   responsibilities  4835 non-null   object 
 4   key_skills        4835 non-null   object 
 5   about_me          4835 non-null   object 
 6   education         4835 non-null   object 
 7   languages         4835 non-null   object 
 8   link              4835 non-null   object 
 9   gender            4835 non-null   int64  
 10  age               4624 non-null   Int64  
 11  experience_years  4835 non-null   Int64  
 12  salary_rub        2922 non-null   Int64  
 13  city              4835 non-null   object 
 14  can_move          4659 non-null   boolean
 15  can_trip          4752 non-null   boolean
dtypes: Int64(3), boolean(2), int64(1), object(

In [ ]:
non_it_keywords = [
    'не указано', 'водитель', 'бухгалтер', 'охранник', 'продавец', 'юрист', 'экономист',
    'секретарь', 'кассир', 'дизайнер', 'трейдер', 'ретушер', 'переводчик',
    'журналист', 'эколог', 'менеджер по продажам', 'smm', 'офис-менеджер'
]

# Отсеиваем не IT опыт
df = df[~df['last_position'].str.lower().apply(lambda x: any(keyword in x for keyword in non_it_keywords))].copy()

df.reset_index(drop=True, inplace=True)

In [ ]:
non_it_keywords = [
    'не указано', 'водитель', 'бухгалтер', 'охранник', 'продавец', 'юрист', 'экономист',
    'секретарь', 'кассир', 'дизайнер', 'трейдер', 'ретушер', 'переводчик',
    'журналист', 'эколог', 'менеджер по продажам', 'smm', 'офис-менеджер', 'cgi',
    'defi', 'спепиалист', 'репетитор', 'хостес', 'ремонт', 'налог', 'техник', 'математик', 'гипсокартон'
]

# Отсеиваем не IT должности
df = df[~df['desired_position'].str.lower().apply(lambda x: any(keyword in x for keyword in non_it_keywords))].copy()

df.reset_index(drop=True, inplace=True)
df.head()

,desired_position,last_position,last_job_company,responsibilities,key_skills,about_me,education,languages,link,gender,age,experience_years,salary_rub,city,can_move,can_trip,job_title_search,category
0,Python-разработчик,Python-разработчик,Диджитал агенство Рядышком,"Создал API сервис, для проекта не имеющего ана...","Python, Django Framework, REST API, DjangoORM,...","Я решил перейти в сферу разработки, так как ме...","ГОУ СПО Строительный колледж №38 Туризм , Мене...","Русский — Родной, Английский — A2 — Элементарный",https://hh.ru/resume/79d001390007c9fa5b0039ed1...,0,32,1,<NA>,Москва,False,True,python-разработчик разработчик,Python
1,Python-разработчик,Python-разработчик,Decurity,Основной задачей была разработка и поддержка с...,"Исполнительность, Асинхронное программирование...",Backend-разработчик с более чем двухлетним опы...,Московский государственный технический универс...,"Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/01e7303a00090f41c50039ed1...,0,26,2,150000,Москва,False,True,python-разработчик разработчик,Python
2,Python-разработчик,Python-разработчик,"Вайлдберриз, ООО",Разрабатывал с коллегами следующие сервисы: Се...,"Python, Django Framework, SQL, Английский язык...",Более 4 лет опыта с python и 2 лет коммерческо...,Московский государственный институт международ...,"Русский — Родной, Английский — C1 — Продвинуты...",https://hh.ru/resume/f4f2bed50005a7d6eb0039ed1...,0,28,5,160000,Москва,False,True,python-разработчик разработчик,Python
3,Python - разработчик,Python-разработчик,"ООО ""МТС ДИДЖИТАЛ""","Разработка бекенда на фреймворке Flask, FastAP...","Python, SQL, Работа в команде, ООП, Django Fra...",Хайбулин Роман Олегович Дата рождения: 17.02.1...,Московский финансово-юридический университет М...,"Русский — Родной, Английский — A2 — Элементарный",https://hh.ru/resume/cd10011e00088d37c50039ed1...,0,26,6,<NA>,Москва,False,True,python - разработчик разработчик,Python
4,Python-разработчик,Разработчик,VK,1. Миграция инфраструктуры (Яндекс → VK) - Зад...,"Python, Telegram, TeamCity, Bash, Docker, Java...",Github: https://github.com/FPSarmin tg:@fpsarmin,"Национальный исследовательский университет ""Вы...","Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/7799fa7000090a8dd80039ed1...,0,23,3,<NA>,Ташкент,True,True,python-разработчик разработчик,Python


In [ ]:
!pip install pymorphy3

In [ ]:
import pandas as pd
import re
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

# 3. Функция предобработки (Лемматизация + Очистка)
def preprocess_title(text):
    if not isinstance(text, str):
        return ''

    text_lower = text.lower()

    # Заменяем все разделители на пробелы ДО очистки.
    # 'web-разработчик' -> 'web разработчик'
    # 'c++/python' -> 'c++ python'
    text_spaced = re.sub(r'[-/\\,]', ' ', text_lower)

    # Очищаем русские слова для лемматизатора
    # (теперь 'web разработчик' -> '   разработчик')
    # (а 'программист с#' -> 'программист с ')
    text_rus_only = re.sub(r'[^а-я\s]', ' ', text_spaced)

    # Лемматизируем
    lemmatized_words = [morph.parse(w)[0].normal_form for w in text_rus_only.split()]
    lemmatized_string = ' '.join(lemmatized_words)

    # Финальная строка для поиска:
    # text_spaced - содержит 'web', 'c#', '.net'
    # lemmatized_string - содержит 'программист', 'разработчик'
    return text_spaced + ' ' + lemmatized_string

# 4. Применяем предобработку
df['job_title_search'] = df['desired_position'].apply(preprocess_title)

# 5. Функция-категоризатор
# 5. ФИНАЛЬНАЯ ВЕРСИЯ. ДОПИЛЕННАЯ.
def categorize_job_regex(title):

    # --- Блок 1: Non-IT ---
    # (Без изменений)
    non_it_pattern = r'\b(водитель|бухгалтер|охранник|продавец|юрист|экономист|секретарь|кассир|дизайнер|трейдер|ретушер|переводчик|журналист|эколог|маркетинг|smm|офис|контент|персонал|закупка|оператор|консультант|юрист|кассир|торговый|представитель|охранник|кладовщик|администратор|помощник|агент|монтажник|слесарь|химик|механик|электрик|электромонтёр|сварщик)\b'
    if re.search(non_it_pattern, title):
        if re.search(r'администратор', title) and re.search(r'\b(системный|сеть|база|бд|it|ит|иб|linux|windows|сисадмин)\b', title):
            pass
        else:
            return 'Non-IT'

    # --- Блок 2: Конкретные Технологии и Роли ---

    if re.search(r'data scientist|data science|\bds\b|\bml\b|machine learning|машинный обучение|data engineer|deep|аналитик данные|bi|computer vision', title):
        return 'Data Science / Data'
    if re.search(r'\b1c\b|\b1с\b', title):
        return '1C'
    if re.search(r'\bqa\b|\btest\b|тестировщик|тестирование|aqa', title):
        return 'QA'
    if re.search(r'devops', title):
        return 'DevOps'
    if re.search(r'security|безопасность|\bиб\b', title):
        return 'InfoSec'

    # !!! НОВАЯ КАТЕГОРИЯ !!! (Тех. поддержка)
    if re.search(r'support|поддержка|helpdesk', title):
        return 'Support / HelpDesk'

    if re.search(r'android|ios|mobile|мобильный|flutter|react native|kotlin|swift|objective-c|xamarin', title):
        return 'Mobile'
    if re.search(r'fullstack|full stack|фуллстек', title):
        return 'Fullstack'

    # (Frontend фреймворки - без изменений)
    if re.search(r'\bangular\b', title):
        return 'Frontend (Angular)'
    if re.search(r'\breact\b', title):
        return 'Frontend (React)'
    if re.search(r'\bvue\b', title):
        return 'Frontend (Vue)'

    # !!! ФИКС ДЛЯ Node.js !!! (ловим 'nodejs' слитно)
    if re.search(r'node\.?js|\bnode\b', title):
        return 'Backend (Node.js)'

    # !!! ФИКС ДЛЯ FRONTEND !!! (ловим 'фронтэнд' и 'fe')
    if re.search(r'\bweb\b|\bвеб\b|front[\s\-]?end|фронтенд|фронтэнд|font[\s\-]?end|frontent|верстальщик|html|css|\bui\b|javascript|\bjs\b|\bfe\b', title):
        return 'Frontend'

    # Python
    if re.search(r'\bpython\b|\bпитон\b', title):
        return 'Python'

    # C# / .NET
    if re.search(r'[cс]\#|\.net', title):
        return 'C# / .NET'

    # Java
    if re.search(r'\bjava\b(?!\s*script)', title):
        return 'Java'

    # Go
    if re.search(r'\bgo\b|\bgolang\b', title):
        return 'Go'

    # C++
    if re.search(r'c\+\+|с\+\+', title):
        return 'C++'

    # PHP
    if re.search(r'\bphp\b', title):
        return 'PHP'

    # Ruby
    if re.search(r'\bruby\b', title):
        return 'Ruby'

    # !!! НОВЫЕ ЯЗЫКИ !!!
    if re.search(r'\brust\b', title):
        return 'Rust'
    if re.search(r'\bclojure\b', title):
        return 'Clojure'
    if re.search(r'\bpascal\b', title):
        return 'Pascal'

    # !!! НОВЫЙ ЯЗЫК 'C' (ВАЖНО: после C++ и C#) !!!
    if re.search(r'\b[cс]\b', title):
        return 'C'

    # DBA / SQL
    if re.search(r'sql|pl/sql|администратор бд|\bбд\b|база данные|oracle|postgres', title):
        return 'DBA / SQL'

    # !!! ФИКС ДЛЯ GAMEDEV !!! (ловим 'unity3d' и т.д.)
    if re.search(r'\bunity|game|\bигра\b|unreal engine', title):
        return 'GameDev'

    # !!! ФИКС ДЛЯ SAP !!! (убираем \b)
    if re.search(r'\bsap|\babap\b', title):
        return 'SAP / ABAP'

    # !!! НОВЫЕ ПЛАТФОРМЫ/ТЕХНОЛОГИИ !!!
    if re.search(r'\bdelphi\b', title):
        return 'Delphi'
    if re.search(r'\bbitrix\b|битрикс', title):
        return 'Bitrix'
    if re.search(r'sharepoint', title):
        return 'SharePoint'
    if re.search(r'\blotus\b', title):
        return 'Lotus'
    if re.search(r'\brpa\b', title):
        return 'RPA'
    if re.search(r'\bflash\b', title):
        return 'Flash'

    # !!! НОВЫЕ ИНЖЕНЕРНЫЕ КАТЕГОРИИ !!!
    if re.search(r'асу|кипиа|плк|plc', title): # АСУ ТП / КИПиА
        return 'АСУ ТП / КИПиА'
    if re.search(r'embedded|встраиваемый|микроконтроллер|микропроцессорный|dsp|qnx|rtos', title): # Embedded
        return 'Embedded'
    if re.search(r'fpga|плис|рэа|схемотехник|электронщик', title): # Hardware
        return 'Hardware / FPGA'


    # --- Блок 3: Общие Роли (когда стек не ясен) ---

    if re.search(r'manager|lead|\b(руководитель|менеджер|teamlead|тимлид|cto|директор|project|архитектор)\b', title):
        return 'Manager / Lead'
    if re.search(r'\b(аналитик|analyst)\b', title):
        return 'System Analyst'
    if re.search(r'системный администратор|\bsysadmin\b|\bсисадмин\b|администратор сеть|linux|windows', title):
        return 'SysAdmin'

    # !!! ФИКС ДЛЯ BACKEND !!! (ловим 'back end' с пробелом)
    if re.search(r'back[\s\-]?end|бэкенд|api|django', title):
        return 'Backend (Other)'

    # --- Блок 4: Все остальные IT-шники ---
    # (Это наша "корзина" для 'Программист', 'Инженер' и т.д.)
    if re.search(r'\b(программист|разработчик|developer|engineer|инженер|it|ит|software|специалист по)\b', title):
        return 'Other IT'

    # --- Блок 5: Все остальное, что не опознали ---
    return 'Other (Unknown)'

# 6. Применяем новую функцию
df['category'] = df['job_title_search'].apply(categorize_job_regex)

# 7. СМОТРИМ РЕЗУЛЬТАТ
print("------ Итоговые категории и их количество: ------")
print(df['category'].value_counts())

# 8. Проверяем "неизвестных"
print("\n\n------ Что попало в 'Other (Unknown)': ------")
unknown_jobs = df[df['category'] == 'Other (Unknown)']['desired_position'].unique()

print(list(unknown_jobs))

------ Итоговые категории и их количество: ------
category
Other IT               2194
Frontend                644
C# / .NET               231
SysAdmin                117
1C                      103
Python                  102
Java                    100
PHP                      82
Fullstack                75
Backend (Other)          70
C++                      69
Non-IT                   55
Mobile                   40
Frontend (React)         35
Other (Unknown)          29
Go                       27
АСУ ТП / КИПиА           21
Manager / Lead           20
Frontend (Angular)       18
Embedded                 18
DBA / SQL                18
Frontend (Vue)           17
Data Science / Data      17
Support / HelpDesk       17
Backend (Node.js)        17
Delphi                   14
GameDev                  14
System Analyst           12
Hardware / FPGA          11
C                        10
Ruby                      7
DevOps                    6
QA                        6
InfoSec          

In [ ]:
other_it = df[df['category'] == 'Other IT']['desired_position'].unique()
other_it

array(['Разработчик', 'Программист-разработчик', 'Программист',
       'Разработчик-программист', 'Ведущий разработчик',
       'Junior программист', 'Программист, разработчик', 'программист',
       'программист-разработчик', 'Инженер-программист', 'Программист-',
       'Программист, разработчик ПО', 'Middle программист',
       'инженер-программист', 'ведущий программист',
       'Разработчик Программист', 'Программист, Информатик',
       'программист (стажёр)', 'Программист на удаленную работу',
       'Начинающий специалист (программист)', 'Ведущий программист',
       'software developer',
       'Ведущий инженер программист встраиваемых систем',
       'Программист \\ разработчик',
       'Подсобный рабочий разнорабочий it-специалист', 'Инженер',
       'специалист программист', 'Специалист по IT',
       'Специалист по системам видеонаблюдения', 'Programmer/developer',
       'Программист / Ведущий программист', 'IT-инженер', 'IT специалист',
       'It-специалист', 'Инженер- 

In [ ]:
df.drop(columns=['job_title_search'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.sample()

,desired_position,last_position,last_job_company,responsibilities,key_skills,about_me,education,languages,link,gender,age,experience_years,salary_rub,city,can_move,can_trip,category
1316,Программист,Инженер-программист,Viburn,"Разработка проектов SCADA-систем, HMI-визуализ...","JavaScript, CSS, HTML, MySQL, CODESYS, SCADA, ...",Готов к изучению любых языков программирования...,Таврический национальный университет им. В.И. ...,"Русский — Родной, Английский — B1 — Средний, У...",https://hh.ru/resume/a8ed3a2000054b4b680039ed1...,0,33,7,<NA>,Симферополь,False,False,Other IT


In [ ]:
# 1. Увеличиваем лимит отображения, чтобы увидеть все 200
pd.set_option('display.max_rows', 200)

# 2. Разворачиваем все скиллы в один столб
all_skills = df['key_skills'].copy().dropna().str.split(r'[,/|]').explode()

# 3. Чистим и считаем
top_skills = all_skills.str.strip().str.lower().value_counts()

# 4. Выводим топ-200
print("------ Топ 200 самых частых скиллов ------")
print(top_skills.head(200))

------ Топ 200 самых частых скиллов ------
key_skills
русский — родной                           4063
git                                        1371
javascript                                 1284
английский — b2 — средне-продвинутый       1280
английский — a1 — начальный                1061
sql                                        1032
html                                       1000
css                                         859
mysql                                       799
html5                                       707
linux                                       704
c++                                         690
php                                         642
ооп                                         641
postgresql                                  593
jquery                                      585
css3                                        584
c#                                          557
python                                      537
английский — b1 — средний         

In [ ]:
# Проведем категоризацию по скилам

# --- Шаг 1: Подготовка колонки 'key_skills' ---
def preprocess_skills(skills):
    if not isinstance(skills, str):
        return ''
    skills_lower = skills.lower()
    # "JavaScript, CSS/HTML" -> "javascript css html"
    skills_spaced = re.sub(r'[,/\\|]', ' ', skills_lower)
    return skills_spaced

df['skills_search'] = df['key_skills'].apply(preprocess_skills)


def categorize_by_skills(skills_str):

    # --- Блок 1: Платформы (CMS, SAP, 1C) ---

    # PHP CMS (до 1С)
    if re.search(r'\bbitrix\b|битрикс|wordpress|joomla', skills_str):
        return 'PHP'

    # 1C
    if re.search(r'\b1c\b|\b1с\b', skills_str):
        return '1C'

    # SAP
    if re.search(r'\bsap\b|\babap\b', skills_str):
        return 'SAP / ABAP'

    # SharePoint
    if re.search(r'sharepoint', skills_str):
        return 'SharePoint'

    # --- Блок 2: Data Science / ML ---
    # (До Python)
    if re.search(r'data scientist|machine learning|keras|pandas|numpy|scikit|tensorflow|pytorch|\bmatlab\b', skills_str):
        return 'Data Science / Data'

    # --- Блок 3: DevOps / SysAdmin ---
    # (До Linux)
    if re.search(r'devops|docker|kubernetes|jenkins|ansible|terraform|\bci\b|\bcd\b', skills_str):
        return 'DevOps'

    if re.search(r'sysadmin|active directory|windows server|администрирование|настройка сервер', skills_str):
        return 'SysAdmin'

    # --- Блок 4: QA ---
    if re.search(r'\bqa\b|selenium|pytest|junit|cypress|playwright|тестирование', skills_str):
        return 'QA'

    # --- Блок 5: Mobile ---
    if re.search(r'android|ios|mobile|flutter|react native|kotlin|swift|objective-c|xamarin', skills_str):
        return 'Mobile'

    # --- Блок 6: Frontend (Фреймворки) ---
    # (До общего JS)
    if re.search(r'\bangular\b', skills_str):
        return 'Frontend (Angular)'
    if re.search(r'\breact\b|\bredux\b|\bnext\.js\b|\bmobx\b|\bjest\b', skills_str):
        return 'Frontend (React)'
    if re.search(r'\bvue\b|\bvuex\b', skills_str):
        return 'Frontend (Vue)'

    # --- Блок 7: Backend (Node.js) ---
    # (До общего JS)
    if re.search(r'node\.?js|\bnode\b', skills_str):
        return 'Backend (Node.js)'

    # --- Блок 8: Frontend (Общий) ---
    if re.search(r'frontend|\bjs\b|javascript|html|css|jquery|bootstrap|typescript|sass|scss|less|webpack|gulp|бэм|верстка', skills_str):
        # Проверка, что это не Java
        if re.search(r'\bjava\b(?!\s*script)', skills_str):
             pass
        else:
             return 'Frontend'

    # --- Блок 9: Backend (Языки) ---
    if re.search(r'\bpython\b|\bdjango\b|\bflask\b|\bfastapi\b|\bcelery\b', skills_str):
        return 'Python'
    if re.search(r'[cс]\#|\.net|\basp\b|entity framework|\bwpf\b|\blinq\b|windows forms', skills_str):
        return 'C# / .NET'
    if re.search(r'\bjava\b(?!\s*script)|\bspring\b|\bhibernate\b|\bmaven\b', skills_str):
        return 'Java'
    if re.search(r'\bphp\b|\blaravel\b|\byii\b', skills_str):
        return 'PHP'
    if re.search(r'\bgo\b|\bgolang\b', skills_str):
        return 'Go'
    if re.search(r'c\+\+|с\+\+|\bqt\b|\bstl\b', skills_str):
        return 'C++'
    if re.search(r'\bruby\b|\brails\b', skills_str):
        return 'Ruby'

    # --- Блок 10: GameDev / Delphi / Pascal ---
    if re.search(r'\bunity\b', skills_str):
        return 'GameDev'
    if re.search(r'\bdelphi\b|object pascal', skills_str):
        return 'Delphi'
    if re.search(r'\bpascal\b', skills_str):
        return 'Pascal'

    # --- Блок 11: Low-level / C ---
    # (После C++ и C#)
    if re.search(r'\bassembler\b', skills_str):
        return 'Embedded'
    if re.search(r'\b[cс]\b', skills_str):
        return 'C'

    # --- Блок 12: Другие IT-роли (низкий приоритет) ---
    if re.search(r'техническая поддержка|support', skills_str):
        return 'Support / HelpDesk'

    # Linux (низкий приоритет, т.к. многие его знают)
    if re.search(r'\blinux\b', skills_str):
        return 'SysAdmin'

    # SQL (самый низкий приоритет, т.к. ВСЕ его знают)
    if re.search(r'sql|mysql|postgresql|mssql|oracle', skills_str):
        return 'DBA / SQL'

    # --- Блок 13: Non-IT (Дизайн и т.п.) ---
    if re.search(r'photoshop|figma|coreldraw|autocad', skills_str):
        return 'Non-IT (Design)'

    # Если не нашли НИЧЕГО из этого - не меняем категорию
    return None

# --- Шаг 3: Применение логики (Без изменений) ---

print("------ СТАТИСТИКА КАТЕГОРИЙ (ДО УТОЧНЕНИЯ): ------")
print(df['category'].value_counts())

# 1. Маска для 'Other IT'
mask_other_it = df['category'] == 'Other IT'

# 2. Применяем функцию по скиллам
new_cats_from_skills = df.loc[mask_other_it, 'skills_search'].apply(categorize_by_skills)

# 3. Обновляем 'category'
df.loc[mask_other_it & new_cats_from_skills.notna(), 'category'] = new_cats_from_skills[new_cats_from_skills.notna()]


print("\n\n------ СТАТИСТИКА КАТЕГОРИЙ (ПОСЛЕ УТОЧНЕНИЯ): ------")
# Ожидаем, что 'Other IT' сильно уменьшится
print(df['category'].value_counts())

print("\n\n------ Что ОСТАЛОСЬ в 'Other IT': ------")
# Тут должны быть 'Программисты' со скиллами типа 'Git, ООП, Agile, HTTP'
remaining_other_it = df[df['category'] == 'Other IT']
print(remaining_other_it[['desired_position', 'key_skills']].head(20))

------ СТАТИСТИКА КАТЕГОРИЙ (ДО УТОЧНЕНИЯ): ------
category
Other IT               2194
Frontend                644
C# / .NET               231
SysAdmin                117
1C                      103
Python                  102
Java                    100
PHP                      82
Fullstack                75
Backend (Other)          70
C++                      69
Non-IT                   55
Mobile                   40
Frontend (React)         35
Other (Unknown)          29
Go                       27
АСУ ТП / КИПиА           21
Manager / Lead           20
Frontend (Angular)       18
Embedded                 18
DBA / SQL                18
Frontend (Vue)           17
Data Science / Data      17
Support / HelpDesk       17
Backend (Node.js)        17
Delphi                   14
GameDev                  14
System Analyst           12
Hardware / FPGA          11
C                        10
Ruby                      7
DevOps                    6
QA                        6
InfoSec         

In [ ]:
# 1. Создаем словарь-маппинг
category_mapping = {
    # --- 1. Frontend ---
    'Frontend (React)': 'Frontend',
    'Frontend (Angular)': 'Frontend',
    'Frontend (Vue)': 'Frontend',
    'Flash': 'Frontend',

    # --- 2. Backend (Niche) ---
    'Backend (Other)': 'Backend',
    'Backend (Node.js)': 'Backend',
    'Go': 'Backend',
    'Ruby': 'Backend',
    'Rust': 'Backend',
    'Clojure': 'Backend',

    # --- 3. Data & Analytics ---
    'Data Science / Data': 'Data & Analytics',
    'DBA / SQL': 'Data & Analytics',
    'System Analyst': 'Data & Analytics',

    # --- 4. Legacy / C ---
    'C': 'Legacy / C',
    'Delphi': 'Legacy / C',
    'Pascal': 'Legacy / C',

    # --- 5. Niche IT ---
    'QA': 'Niche IT',
    'Support / HelpDesk': 'Niche IT',
    'Embedded': 'Niche IT',
    'АСУ ТП / КИПиА': 'Niche IT',
    'SAP / ABAP': 'Niche IT',
    'Manager / Lead': 'Niche IT',
    'GameDev': 'Niche IT',
    'Hardware / FPGA': 'Niche IT',
    'SharePoint': 'Niche IT',
    'InfoSec': 'Niche IT',
    'RPA': 'Niche IT',
    'Lotus': 'Niche IT',

    # --- 6. Non-IT / Unknown ---
    'Non-IT': 'Non-IT / Unknown',
    'Other (Unknown)': 'Non-IT / Unknown',
    'Non-IT (Design)': 'Non-IT / Unknown',

    # --- 7. PHP ---
    'Bitrix': 'PHP'
}

# 2. Применяем маппинг
df['category'] = df['category'].replace(category_mapping)

# 3. СМОТРИМ ФИНАЛЬНЫЙ РЕЗУЛЬТАТ
print("------ СТАТИСТИКА КАТЕГОРИЙ: ------")
print(df['category'].value_counts())

------ СТАТИСТИКА КАТЕГОРИЙ: ------
category
Frontend            946
Other IT            817
C# / .NET           324
1C                  318
SysAdmin            273
PHP                 183
Python              173
Niche IT            171
Backend             149
Java                140
C++                 132
Data & Analytics    127
Legacy / C          122
Mobile              112
Non-IT / Unknown    101
Fullstack            75
DevOps               73
Name: count, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4236 entries, 0 to 4235
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   desired_position  4236 non-null   object 
 1   last_position     4236 non-null   object 
 2   last_job_company  4236 non-null   object 
 3   responsibilities  4236 non-null   object 
 4   key_skills        4236 non-null   object 
 5   about_me          4236 non-null   object 
 6   education         4236 non-null   object 
 7   languages         4236 non-null   object 
 8   link              4236 non-null   object 
 9   gender            4236 non-null   int64  
 10  age               4053 non-null   Int64  
 11  experience_years  4236 non-null   Int64  
 12  salary_rub        2552 non-null   Int64  
 13  city              4236 non-null   object 
 14  can_move          4080 non-null   boolean
 15  can_trip          4161 non-null   boolean
 16  category          4236 non-null   object 
